In [ ]:
import pandas as pd
import covid19_module

In [ ]:
df_cases_raw = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", index_col=False)
df_deaths_raw = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", index_col=False)
df_recoveries_raw = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv", index_col=False)
df_population_raw = pd.read_csv("./population_data/API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv")

df_cases = covid19_module.preprocess_frame(df_cases_raw)
df_deaths = covid19_module.preprocess_frame(df_deaths_raw)
df_recoveries = covid19_module.preprocess_frame(df_recoveries_raw)
df_population = df_population_raw[["Country Name", "2018"]]

covid19_module.shift_to_day_zero(df_cases, df_cases)
covid19_module.shift_to_day_zero(df_deaths, df_deaths)
covid19_module.shift_to_day_zero(df_recoveries, df_recoveries)

In [ ]:
top_cases = covid19_module.top_countries(df_cases_raw)
top_deaths = covid19_module.top_countries(df_deaths_raw)

covid19_module.plot_top_countries(df_cases, top_cases, "confirmed")
covid19_module.plot_top_countries(df_deaths, top_deaths, "deaths")

#covid19_module.print_mortality_rates(df_cases, df_deaths, top_deaths)
#covid19_module.print_percentage_infected(df_cases, df_population, top_cases)
#covid19_module.print_permil_deaths(df_deaths, df_population, top_deaths)

In [ ]:
#%matplotlib notebook
covid19_module.plot_growth(df_cases, top_cases, "confirmed")
covid19_module.plot_growth(df_deaths, top_deaths, "deaths")

In [ ]:
#%matplotlib notebook
covid19_module.plot_case_death_recovery('China', df_cases, df_deaths, df_recoveries)
covid19_module.plot_case_death_recovery('notChina', df_cases, df_deaths, df_recoveries)

In [ ]:
top_countries = top_cases.tolist()
top_countries.append("Greece")
for country in top_countries:
    try:
        log_popt, log_pcov, inflection_p = covid19_module.fit_logistic(country, df_cases)
        fit_end = inflection_p[1]
        if fit_end > df_cases[country].dropna().shape[0]:
            fit_end = df_cases[country].dropna().shape[0]
        exp_popt, exp_pcov, R2 = covid19_module.fit_exponential(country, df_cases, [0,fit_end])
        covid19_module.plot_fits(country, df_cases, exp_popt, exp_pcov, log_popt, log_pcov, inflection_p[0], "confirmed")
    except:
        pass

In [ ]:
top_countries = top_deaths.tolist()
top_countries.append("Greece")
for country in top_countries:
    try:
        log_popt, log_pcov, inflection_p = covid19_module.fit_logistic(country, df_deaths)
        fit_end = inflection_p[1]
        if fit_end > df_deaths[country].dropna().shape[0]:
            fit_end = df_deaths[country].dropna().shape[0]
        exp_popt, exp_pcov, R2 = covid19_module.fit_exponential(country, df_deaths, [0,fit_end])
        covid19_module.plot_fits(country, df_deaths, exp_popt, exp_pcov, log_popt, log_pcov, inflection_p[0], "confirmed")
    except:
        pass

In [ ]:
#%matplotlib notebook
interval = 14
for country in top_cases:
    covid19_module.plot_daily_vs_total(df_cases, country, interval)

In [ ]:
#%matplotlib notebook
df_daily = covid19_module.add_daily_entries(df_cases)
covid19_module.fit_normal(df_daily, "China")

In [ ]:
%matplotlib notebook
import uncertainties as unc
import uncertainties.unumpy as unp

import numpy as np
from scipy.stats import t
import matplotlib.pyplot as plt


country = "US"

x = np.linspace(0, df_cases[country].dropna().shape[0] + 5, 100)
y = covid19_module.logistic(x, *log_popt) 

inflection_p_idx = covid19_module.inflection_point(x,y)

# Graph
plt.plot(x, covid19_module.logistic(x, *log_popt))
plt.plot(x[inflection_p_idx], y[inflection_p_idx], 'or', label='estimated inflection point')
plt.xlabel('x'); plt.ylabel('y'); plt.legend();

#fit_end = covid19_module.index_of_max_R2(df_cases, country)
fit_end = np.floor(x[inflection_p_idx])
exp_popt, exp_pcov, R = covid19_module.fit_exponential(country, df_cases, [0, fit_end])
log_popt, log_pcov = covid19_module.fit_logistic(country, df_cases)
covid19_module.plot_fits(country, df_cases, exp_popt, exp_pcov, log_popt, log_pcov, "confirmed")



#plt.yscale("log")

In [ ]:
np.floor(x[idx_max_dy])